# Pagalbinės funkcijos

In [1]:
from Bio import SeqIO

def isStopCodon(codon):
    stop_codons = ['TAG', 'TAA', 'TGA']
    isStopCodon = False
    for stop_codon in stop_codons:
        if codon.find(stop_codon) != -1:
            isStopCodon = True
    return isStopCodon

def isStartCodon(codon):
    if codon.find("ATG") != -1:
        return True
    return False

In [2]:
import numpy as np
import pandas as pd

def ratesToTable(all_rates, file_name):
    if(len(all_rates) == 0):
        return    
    col_names = []
    row_names = []
    matrix = np.empty(shape=[len(all_rates), len(all_rates[0])-1])
    for row, rates in enumerate(all_rates):
        row_names.append(rates[0])
        for col, (name, rate1) in enumerate(rates[1:]):
            matrix[row][col] = rate1
            if(row == 0):
                col_names.append(name)
    with open("viruses\\results\\{}.csv".format(file_name), "w") as file:
        file.write(pd.DataFrame(data = matrix, index=row_names, columns=col_names).to_csv())  
    file.close()      

In [3]:
def allCodons():
    letters = "ATCG"
    all_codons = []
    for c1 in letters:
        for c2 in letters:
            for c3 in letters:
                all_codons.append(c1+c2+c3)
    return all_codons

In [4]:
def allDicodons(all_codons):
    all_dicodons = []
    for codon1 in all_codons:
        for codon2 in all_codons:
            all_dicodons.append(codon1 + codon2)
    return all_dicodons

In [5]:
def csToString(coding_sequences):
    coding_sequences_string = ""
    for coding_sequence in coding_sequences:
        coding_sequences_string += coding_sequence
    return coding_sequences_string

# Koduojančių sekų paieška

In [6]:
from Bio import SeqIO

def findCodingSequences(file_name):
    fileLoc = "viruses/data/{}.fasta"
    record = SeqIO.read(fileLoc.format(file_name), "fasta")
    sequence = record.seq
    coding_sequences_temp = []
    coding_sequences = []
    codon_count = 0
    for sequence in [record.seq, record.seq.reverse_complement()]:
        for frame in range(3):
            length = (len(record)-frame)
            codons = [sequence[frame+i:frame+i+3] for i in range (0, length, 3)]
            frame_coding_sequences = []
            orf = ""
            write = False
            for codon in codons:
                if isStartCodon(codon):
                    write = True
                if isStopCodon(codon) and write:
                    orf = orf + codon
                    if(len(orf) > 100):
                        frame_coding_sequences.append(orf)
                    orf = ""
                    write = False
                elif write:
                    orf = orf + codon
            coding_sequences_temp.append(frame_coding_sequences)
            with open("viruses\\results\\{}_coding_sequences.txt".format(file_name), "w") as file:
                file.write("{}\n".format(record.id))
                for frame_coding_sequences in coding_sequences_temp:
                    for coding_sequence in frame_coding_sequences:
                        file.write("{}\n".format(coding_sequence))
                        coding_sequences.append(coding_sequence)
                        codon_count += len(coding_sequence)/3
    file.close()
    return (record.id, coding_sequences, codon_count)  

# Dažnių skaičiavimas

In [7]:
def findCodonRate(id_codingSequences, all_codons):
    (id, coding_sequences, length) = id_codingSequences
    codon_rates = [id]
    coding_sequences_string = csToString(coding_sequences)
    for codon in all_codons:
        codon_rates.append((codon,coding_sequences_string.count(codon)/length))
    return codon_rates


def findDicodonRate(id_codingSequences_len, all_dicodons):
    (id, coding_sequences, length) = id_codingSequences_len
    dicodon_count_dict = {}
    dicodons_count = 0
    dicodon_rates = [id]

    for dicodon in all_dicodons:
        dicodon_count_dict[dicodon] = 0
    for coding_sequence in coding_sequences:
        dicodons = [coding_sequence[i:i+6] for i in range (0, len(coding_sequence), 3)]
        for dicodon in dicodons:
            if(dicodon_count_dict.get(dicodon) != None):
                dicodons_count += 1
                dicodon_count_dict[dicodon] += 1
    for (name, count) in dicodon_count_dict.items():
        dicodon_rates.append((name,count/dicodons_count))
    return dicodon_rates

# Atstumo skaičiavimas

In [8]:
import math
import numpy as np
import pandas as pd

def countDistances(all_rates, file_name):
    matrix = np.empty(shape =[len(all_rates), len(all_rates)])
    row_names = []
    for row, rates1 in enumerate(all_rates):
        row_names.append(rates1[0])
        for col, rates2 in enumerate(all_rates):
            sum = 0
            for idx, (_,rate1) in enumerate(rates1[1:], start = 1):
                (_, rate2) = rates2[idx]
                sum += (rate1 - rate2)**2
            matrix[row][col] = math.sqrt(sum)
    with open("viruses\\results\\{}_distances.txt".format(file_name), "w") as file:
        file.write(pd.DataFrame(matrix, index=row_names).to_string())
    file.close()    

# Pagrindinė funkcija

In [9]:
file_name_array = []

bacterial1 = "bacterial1"
bacterial2 = "bacterial2"
bacterial3 = "bacterial3"
bacterial4 = "bacterial4"
mamalian1 = "mamalian1"
mamalian2 = "mamalian2"
mamalian3 = "mamalian3"
mamalian4 = "mamalian4"

file_name_array.append(bacterial1)
file_name_array.append(bacterial2)
file_name_array.append(bacterial3)
file_name_array.append(bacterial4)
file_name_array.append(mamalian1)
file_name_array.append(mamalian2)
file_name_array.append(mamalian3)
file_name_array.append(mamalian4)

all_codons = allCodons()
all_dicodons = allDicodons(all_codons)
all_coding_sequences = []
all_codon_rates = []
all_dicodon_rates = []

for file_name in file_name_array:
    all_coding_sequences.append(findCodingSequences(file_name))

for coding_sequences in all_coding_sequences:
    all_codon_rates.append(findCodonRate(coding_sequences, all_codons))
    all_dicodon_rates.append(findDicodonRate(coding_sequences, all_dicodons))


countDistances(all_codon_rates, "codons_rate")
countDistances(all_dicodon_rates, "dicodons_rate")
ratesToTable(all_codon_rates, "codons_rates")
ratesToTable(all_dicodon_rates, "dicodons_rates")